In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [3]:
import pandas as pd
import numpy as np

installs = pd.read_csv(r'C:\Users\Matias\Documents\UBA\Datos\installs.csv.gzip',compression = 'gzip')

In [4]:
types = {'date':'object','event_id':'int64','ref_type':'category','ref_hash':'int64','application_id':'int64',\
         'attributed':'bool','device_countrycode':'category','device_os_version':'float64','device_brand':'float64',\
         'device_model':'float64','device_city':'float64','session_user_agent':'float64','trans_id':'category',\
         'user_agent':'float64','event_uuid':'object','carrier':'float64','kind':'float64','device_os':'category',\
         'wifi':'bool','connection_type':'category','ip_address':'int64','device_language':'float64'}

cols = ['date','event_id','ref_type','ref_hash','application_id','attributed','device_os_version','device_brand','device_model'
       ,'device_city','session_user_agent','trans_id','user_agent','event_uuid','carrier','kind','device_os','wifi','connection_type'
       ,'ip_address','device_language']

events = pd.read_csv(r'C:\Users\Matias\Documents\UBA\Datos\events.csv.gzip',usecols = cols,dtype = types,compression = 'gzip')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:

#PRE: 
#df['timestamp'] = timestamp de una fecha
#date fecha a comparar
#refhash para filtrar los events

#POS:
#devuelve 2 dataframes, para checkear si estan vacios usar df.empty

def find_lowest_neighbour_in_time(df,date,refhash):
    df = df[df['ref_hash'] == refhash]
    exactmatch=df[df['date']==date]
    lowerneighbour_ind = df[df['timestamp']<date.timestamp()].nlargest(1,'timestamp')
    return lowerneighbour_ind,exactmatch

In [6]:
events['date'] = pd.to_datetime(events['date'])

installs['created'] = pd.to_datetime(installs['created'])
installs['hour'] = installs['created'].apply(lambda x: x.hour)
installs['hour'] = installs['hour'].astype('int8')

installs['day'] = installs['created'].apply(lambda x: x.day)
installs['day'] = installs['day'].astype('int8')

installs['install_value'] = 1

In [7]:
installs['origin_event_uuid'] = None
installs['exact_event_uuid'] = None

In [8]:
events['timestamp'] = events['date'].apply(lambda x: x.timestamp())
events['timestamp'] = events['timestamp'].astype('float32')
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494423 entries, 0 to 2494422
Data columns (total 22 columns):
date                  datetime64[ns]
event_id              int64
ref_type              category
ref_hash              int64
application_id        int64
attributed            bool
device_os_version     float64
device_brand          float64
device_model          float64
device_city           float64
session_user_agent    float64
trans_id              category
user_agent            float64
event_uuid            object
carrier               float64
kind                  float64
device_os             category
wifi                  object
connection_type       category
ip_address            int64
device_language       float64
timestamp             float32
dtypes: bool(1), category(4), datetime64[ns](1), float32(1), float64(9), int64(4), object(2)
memory usage: 325.9+ MB


In [9]:
installsByTime = installs.groupby('created').agg({'install_value':sum,'application_id':lambda x: x,'ref_hash':lambda x: x})
installsDevice = installs.groupby('ref_hash').agg({'install_value':sum,'application_id':lambda x: x,'created':lambda x: x})
installsByTime.head()

,install_value,application_id,ref_hash
created,,,
2019-03-05 00:00:38.219,1,5,5924583283197158397
2019-03-05 00:05:35.234,1,16,1740562126672148647
2019-03-05 00:07:25.431,1,16,760677655438420566
2019-03-05 00:07:38.572,1,8,5006759053482448963
2019-03-05 00:10:36.708,1,7,3318538943125451633


In [13]:
date = installsByTime.index.tolist()[0]
neighbour,exact = find_lowest_neighbour_in_time(events,date,installsByTime.iloc[0]['ref_hash'])
neighbour #.empty

,date,event_id,ref_type,ref_hash,application_id,attributed,device_os_version,device_brand,device_model,device_city,...,user_agent,event_uuid,carrier,kind,device_os,wifi,connection_type,ip_address,device_language,timestamp


In [14]:
for date in installsByTime.index.tolist():
    neighbour,exact = find_lowest_neighbour_in_time(events,date,installsByTime.loc[date,'ref_hash'])
    if neighbour.empty == False:
        install = installs[installs['created'] == date]
        value = neighbour.loc[neighbour.index[0],'event_uuid']
        installs.loc[install.index[0],'origin_event_uuid'] = value
    if exact.empty == False:
        install = installs[installs['created'] == date]
        value = exact.loc[exact.index[0],'event_uuid']
        installs.loc[install.index[0],'exact_event_uuid'] = value
        

In [15]:
installs.head()

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,...,kind,wifi,trans_id,ip_address,device_language,hour,day,install_value,origin_event_uuid,exact_event_uuid
0,2019-03-13 01:43:33.445,0,1891515180541284343,8464844987297247076,NaN,False,False,6333597102633388268,3.083059e+17,4.445252e+18,...,NaN,NaN,NaN,5529284367937637301,NaN,1,13,1,None,None
1,2019-03-13 04:58:35.078,0,1891515180541284343,3250564871270161533,NaN,False,False,6333597102633388268,3.083059e+17,4.445252e+18,...,NaN,NaN,NaN,6098022721363533304,NaN,4,13,1,None,None
2,2019-03-13 04:20:57.666,0,1891515180541284343,7953318831018100268,NaN,False,True,6333597102633388268,5.137992e+17,6.026543e+18,...,Open,NaN,NaN,4636407223847323867,3.301378e+18,4,13,1,None,0bfa9b4c-e07a-4522-b32d-138e3165ac98
3,2019-03-13 04:20:57.698,0,1891515180541284343,7953318831018100268,NaN,False,False,6333597102633388268,5.137992e+17,6.026543e+18,...,NaN,NaN,NaN,4636407223847323867,NaN,4,13,1,None,None
4,2019-03-10 22:24:56.684,0,1891515180541284343,8355110941557237501,NaN,False,True,6333597102633388268,1.083369e+18,7.876612e+18,...,Open,NaN,NaN,685400811232710215,3.301378e+18,22,10,1,None,8f235a8c-41bf-4399-b33d-c41577c949b0


In [16]:
ref = 7953318831018100268
events[events['ref_hash'] ==ref]

,date,event_id,ref_type,ref_hash,application_id,attributed,device_os_version,device_brand,device_model,device_city,...,user_agent,event_uuid,carrier,kind,device_os,wifi,connection_type,ip_address,device_language,timestamp
3333,2019-03-13 04:20:57.666,31,1891515180541284343,7953318831018100268,0,False,2.635068e+18,5.137992e+17,6.026543e+18,8.578627e+17,...,NaN,0bfa9b4c-e07a-4522-b32d-138e3165ac98,7.216102e+18,8.820572e+18,NaN,NaN,NaN,4636407223847323867,3.301378e+18,1.552451e+09


In [17]:
installsByTime.iloc[0]

install_value                       1
application_id                      5
ref_hash          5924583283197158397
Name: 2019-03-05 00:00:38.219000, dtype: int64